# Exercise: Create a Python Package for Cloud-Free Mosaics

In this exercise, your goal is to:

1. Create a function that generates a cloud-free mosaic using Sentinel-2 data and Google Earth Engine.
2. Turn this function into a reusable Python package.
3. Organize your code using proper Python package structure.
4. Install your package locally in editable mode.
5. Test your package using a script or notebook.

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-kamilmolo13')

aoi = ee.Geometry.Point([23.787745, 50.903603])


Successfully saved authorization token.


## Step 1: Write the Function
Create a function called `create_cloud_free_mosaic(aoi, start_date, end_date)` that:
- Filters Sentinel-2 imagery by date and area,
- Applies a cloud mask using the `QA60` band,
- Returns a cloud-free median composite.

In [7]:
def create_cloud_free_mosaic(aoi, start_date, end_date):
    image_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi) \

    def cloud_mask(image):
        qa_band = image.select('QA60')
        cloud_bit = 1 << 10
        cirrus_bit = 1 << 11
        c_mask = qa_band.bitwiseAnd(cloud_bit).eq(0).And(qa_band.bitwiseAnd(cirrus_bit).eq(0))
        return image.updateMask(c_mask).copyProperties(image, image.propertyNames())
    
    mosaic = image_collection.map(cloud_mask).median().clip(aoi)

    return mosaic

april_images = create_cloud_free_mosaic(aoi=aoi, start_date='2023-04-01', end_date='2023-05-01')

## Step 2: Create the Package Structure
Use the following folder layout:

```
geoapps/
├── geoapps/
│   ├── __init__.py
│   └── lab03/
│       ├── __init__.py
│       └── cloud_mosaic.py  # <- your function goes here
├── pyproject.toml
└── README.md
```

## Step 3: Create `pyproject.toml`
Use the following template:

```toml
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "geoapps"
version = "0.1.0"
requires-python = ">=3.10"
description = "EO utilities for cloud-free mosaics"
authors = [
  { name = "Your Name", email = "your@email.com" }
]

[tool.setuptools.packages.find]
where = ["."]
exclude = ["tests", "notebooks"]
```

## Step 4: Install Your Package
From the root of your project (where `pyproject.toml` is located), run:

```bash
pip install -e .
```

## Step 5: Test the Package
Create a test script or notebook and try importing your function like this:

```python
from geoapps.lab03.cloud_mosaic import create_cloud_free_mosaic

import ee
ee.Initialize(project='your-ee-project-id')

aoi = ee.Geometry.Rectangle([19.8, 50.0, 20.0, 50.2])
mosaic = create_cloud_free_mosaic(aoi, '2023-06-01', '2023-07-01')
print(mosaic.getInfo())
```

## Step 6: Bonus – Add a Demo Notebook
Create a notebook in a separate `notebooks/` folder where you import and visualize the result using `geemap`:

```python
import geemap
Map = geemap.Map(center=[50.1, 19.9], zoom=10)
Map.addLayer(mosaic, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Mosaic')
Map
```